In [4]:
# pip install snowflake-connector-python
# pip install load_dotenv

import os
# import findspark
# findspark.init()
from dotenv import load_dotenv
from pyspark.sql import SparkSession
load_dotenv()

# Initialize a Spark session
spark = (SparkSession.builder
    .appName("Snowflake to PySpark")
    .config("spark.jars.packages", "net.snowflake:spark-snowflake_2.12:2.10.0-spark_3.2") 
    .master("local[*]")
    .getOrCreate())

# Display the Spark session 
spark

24/08/10 18:25:41 WARN Utils: Your hostname, codespaces-7c15f5 resolves to a loopback address: 127.0.0.1; using 10.0.2.28 instead (on interface eth0)
24/08/10 18:25:41 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Ivy Default Cache set to: /home/codespace/.ivy2/cache
The jars for the packages stored in: /home/codespace/.ivy2/jars
:: loading settings :: url = jar:file:/usr/local/python/3.10.13/lib/python3.10/site-packages/pyspark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
net.snowflake#spark-snowflake_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-735ea9fa-6533-4abf-90cf-ea2e2db397d0;1.0
	confs: [default]
	found net.snowflake#spark-snowflake_2.12;2.10.0-spark_3.2 in central
	found net.snowflake#snowflake-ingest-sdk;0.10.3 in central
	found net.snowflake#snowflake-jdbc;3.13.14 in central
downloading https://repo1.maven.org/maven2/net/snowflake/spark-snowflake_2.12/2.10.0-spark_3.2/spark-snowflake_

In [5]:
# Snowflake connection options
sfOptions = {
    "sfURL": f"https://{os.getenv('ACCOUNT')}.snowflakecomputing.com",
    "sfUser": os.getenv('USER_NAME'),
    "sfPassword": os.getenv('PASSWORD'),
    "sfDatabase": "TEST",
    "sfSchema": "TEST",
    "sfWarehouse": "compute_wh",
    "sfRole": "ACCOUNTADMIN"
}

# Read the data from Snowflake table into a DataFrame
df = spark.read \
    .format("net.snowflake.spark.snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "COVID19_EPIDEMIOLOGICAL_DATA.PUBLIC.DEMOGRAPHICS") \
    .load()

# Register the DataFrame as a temporary SQL view for processing
df.createOrReplaceTempView("demographics")


24/08/10 18:26:12 WARN SnowflakeConnectorUtils$: Query pushdown is not supported because you are using Spark 3.0.1 with a connector designed to support Spark 3.2. Either use the version of Spark supported by the connector or install a version of the connector that supports your version of Spark.


In [6]:
df.show()

+---------+---------+-----+----------+------------+-----+-----------------+----------------+---------------------+-----------------------+
|ISO3166_1|ISO3166_2| FIPS|  LATITUDE|   LONGITUDE|STATE|           COUNTY|TOTAL_POPULATION|TOTAL_MALE_POPULATION|TOTAL_FEMALE_POPULATION|
+---------+---------+-----+----------+------------+-----+-----------------+----------------+---------------------+-----------------------+
|       US|       TX|48203|32.5036285| -94.0554582|   TX|  Harrison County|           66431|                32573|                  33858|
|       US|       TX|48205|35.7167664|-102.3805857|   TX|   Hartley County|            5966|                 3596|                   2370|
|       US|       TX|48207| 33.120371| -99.6317724|   TX|   Haskell County|            5812|                 3236|                   2576|
|       US|       TX|48209|30.0241697| -97.7473311|   TX|      Hays County|          185686|                92221|                  93465|
|       US|       TX|48211|

In [8]:
df.count()

3140

In [9]:
df.write.format("snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "TEST.TEST.DEMOGRAPHICS_TEST") \
    .mode("overwrite") \
    .save()

24/08/10 16:29:28 WARN SnowflakeConnectorUtils$: Query pushdown is not supported because you are using Spark 3.0.1 with a connector designed to support Spark 3.2. Either use the version of Spark supported by the connector or install a version of the connector that supports your version of Spark.


24/08/10 16:29:32 WARN SnowflakeConnectorUtils$: Query pushdown is not supported because you are using Spark 3.0.1 with a connector designed to support Spark 3.2. Either use the version of Spark supported by the connector or install a version of the connector that supports your version of Spark.


In [12]:
spark.read \
    .format("net.snowflake.spark.snowflake") \
    .options(**sfOptions) \
    .option("dbtable", "TEST.TEST.DEMOGRAPHICS_TEST") \
    .load().count()

24/08/10 16:31:14 WARN SnowflakeConnectorUtils$: Query pushdown is not supported because you are using Spark 3.0.1 with a connector designed to support Spark 3.2. Either use the version of Spark supported by the connector or install a version of the connector that supports your version of Spark.


3140